<a href="https://colab.research.google.com/github/r-doz/PML2025/blob/main/./Homeworks/Homework_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 2

Probabilistic Machine Learning -- Spring 2025, UniTS

### Problem 1: Marginal Likelihood Optimisation

Given the Bayesian Linear Regression model implemented in the Notebook 6, considering the same dataset and variables (Apparent Temperature vs Humidity), optimize alpha and beta by maximizing the Marginal Likelihood.

Note: You can show here only the piece of code that you used and write the optimal alpha and beta that you obtained (so you can run it in directly on Notebook 6)

Hint: import scipy.optimize as optimize

### Problem 2: Rejection Sampling

Consider the unnormalized probability density function:

$$
\tilde{p}(x) = \exp\left(-\frac{x^4}{4} - \frac{x^2}{2}\right)
$$

This density is defined for $x \in \mathbb{R}$, but it is not normalized. Let $Z$ be its normalization constant:

$$
Z = \int_{-\infty}^{\infty} \tilde{p}(x)\, dx
$$

You are given access to a proposal distribution $q(x) = \mathcal{N}(0, 1)$ from which you can sample and evaluate its density.

-  Implement and then use rejection sampling with $q(x)$ to generate samples from the normalized target distribution $p(x) = \frac{1}{Z} \tilde{p}(x)$.
    
- Estimate the normalization constant $Z$ (hint: see your course notes!)

- Compare your result with a numerical approximation of $Z$ using integration methods (e.g., scipy.integrate.quad).


In [ ]:
import numpy as np
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.integrate import quad

In [ ]:
def rejection_sampling(target_pdf, proposal_pdf, proposal_sampler, n_samples):
    samples = []
    max_ratio = max(target_pdf(x) / proposal_pdf(x) for x in np.arange(-10, 10, 0.01)) # Adjust range as per your distribution
    #print(f"max_ratio: {max_ratio}")
    i =0 
    
    while len(samples) < n_samples:
        x = proposal_sampler()  # Sample from proposal distribution
        #print(f"x numero: {i+1}")
        i = i + 1
        u = np.random.uniform(0, max_ratio * proposal_pdf(x))
        
        if u <= target_pdf(x):
            #print('entrato')
            samples.append(x)
        else:
            print('non entrato')

    print(i)
    P_accepted = 1 - (i - len(samples)) / i
    print(f"Probability of being accepted: {P_accepted}")
    print(f"partition function: {P_accepted * max_ratio}")


    return samples

# Example usage:
def target_pdf(x):
    return np.exp((-x**4/4) - (x**2/2))    # Example target distribution, e.g., Gaussian
def proposal_pdf(x):
    return norm.pdf(x, loc=0, scale=1)    # Example proposal distribution, e.g., Exponential

samples = rejection_sampling(target_pdf, proposal_pdf,  np.random.normal, n_samples=10000)

Z, error = quad(target_pdf, -np.inf, np.inf)

print("Valore dell'integrale Z:", Z)
print("Errore stimato:", error)

x = np.linspace(-4, 4, 1000)
plt.plot(x, target_pdf(x), label="target_pdf")
plt.plot(x, proposal_pdf(x), label="proposal_pdf")
#plt.plot(x, 18.521616940414205 * proposal_pdf(x), label="proposal_pdf (norm)")
_ = sns.histplot(samples, kde=True, stat='density', label='Normalized_target_pdf')
_ = plt.legend()

In [ ]:
print(len(samples))

### Problem 3: MCMC Convergence Diagnostics

You have implemented a Metropolis-Hastings algorithm and used it to draw samples from a univariate target distribution. In this exercise, you will compute convergence diagnostics from scratch to assess whether your chains have mixed well.

Consider 4 parallel chains, each of length $N$ (after burn-in), starting from different initial values (use
initial_values = [-10, -2, 2, 10]). The unnormalized probability density function $\tilde{p}(x)$ is defined as a mixture of two Gaussian distributions:

$$
\tilde{p}(x) = \mathcal{N}(x; -5, 1) + \mathcal{N}(x; 5, 1)
$$

Perform the following steps two times, considering a proposal standard deviation of 0.1 and 2.0:

- Compute the within variance $W$ and the between variance $B$
- Compute the statistics $\hat{R}$
- For a single chain of samples $x_1, x_2, \dots, x_N$, compute the lag-$k$ autocorrelation $\rho_k$ and plot $\rho_k$ for $k = 1, 2, \dots, 20$
- Estimate the effective number of samples $n_{eff}$

Repeat the analysis with the distribution:

$$
\tilde{p}(x) = \mathcal{N}(x; -2, 1) + \mathcal{N}(x; 2, 1)
$$

Discuss the results



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pyro
import pyro.distributions as dist
from pyro.infer.mcmc import MCMC, HMC, NUTS
import torch
import seaborn as sns
import statsmodels.api as sm
from statsmodels.graphics import tsaplots


In [ ]:
def sample_from_q(x, sigma):
    #your code here
    return np.random.normal(x, sigma)

def q_y_given_x(y, x, sigma):
    #your code here
    return stats.norm(x, sigma).pdf(y)

def p_tilde(x):
    #your code here
    return stats.norm(-2, 1).pdf(x) + stats.norm(2, 1).pdf(x) 

def metropolis_hastings(x_0, n, sigma):
    rejections = 0
    x = np.zeros(n)
    x[0] = x_0
    print(f"Inizio da x[0] = {x[0]}")
    for i in range(1, n):
        current_x = x[i-1]
        y = sample_from_q(current_x, sigma)

        print(f"\nStep {i}:")
        print(f"  Proposto y = {y}")
        print(f"  Punto attuale x = {current_x}")

        alpha = min(1., p_tilde(y)/ p_tilde(current_x) * q_y_given_x(current_x, y, sigma) / q_y_given_x(y, current_x, sigma))

        print(f"alpha: {alpha} \n 1^:{ p_tilde(y)/ p_tilde(current_x)} \n 2^: = {q_y_given_x(current_x, y, sigma) }   { q_y_given_x(y, current_x, sigma)}")

        if np.random.rand()  < alpha:
            print("  >>> Accettato")
            x[i] = y
        else:
            print("  >>> Rifiutato")
            x[i] = current_x
            rejections += 1

    return x, rejections

def within_variance(m, n, initial_value, sigma):
    initial_value = np.asarray(initial_value)  
    burn_in = int(n/2)

    if len(initial_value) != m:
        raise ValueError(f"{initial_value} Incompatible length")
    
    S = []
    mean  = []
    
    for x_0 in initial_value:
        x,_ = metropolis_hastings(x_0, n, sigma)
        x = x[burn_in:]
        mean = np.append(mean, np.mean(x))
        s = sum((x - mean[-1]) ** 2) / (n - 1)
        S = np.append(S, s)

    within_variance = sum(S)/m
    between_variance = sum((mean - np.mean(mean)) ** 2) * (n / (m - 1))
    upper_buond = ((n - 1) / m) * within_variance + (1 / n) * between_variance
    R = np.sqrt(upper_buond / within_variance)

    return print(f'within variance: {within_variance} \n between variance: {between_variance} \n upper bound: {upper_buond} \n R: {R}')

    

t = np.linspace(-10, 10, 1000)
plt.plot(t, p_tilde(t), label='p_tilde')
plt.legend()
plt.show()

In [ ]:
within_variance(4, 1000, [-10, -2, 2, 10], 2)
 

In [ ]:

burn_in = 500 # number of samples to discard before collecting data
n = 1000

x_1,r_1 = metropolis_hastings(-10, n, sigma = 0.1)
x_2,r_2 = metropolis_hastings(-2, n, sigma = 0.1)
x_3,r_3 = metropolis_hastings(2, n, sigma = 0.1)  
x_4,r_4 = metropolis_hastings(10, n, sigma = 0.1)

y_1,s_1 = metropolis_hastings(-10, n, sigma = 2)
y_2,s_2 = metropolis_hastings(-2, n, sigma = 2)
y_3,s_3 = metropolis_hastings(2, n, sigma = 2)  
y_4,s_4 = metropolis_hastings(10, n, sigma = 2)

x_1 = x_1[burn_in:]
x_2 = x_2[burn_in:]
x_3 = x_3[burn_in:]
x_4 = x_4[burn_in:]

y_1 = y_1[burn_in:]
y_2 = y_2[burn_in:]
y_3 = y_3[burn_in:]
y_4 = y_4[burn_in:]

# print('x:', len(x))
# print('Rejection rate:', r)


In [ ]:

fig, axs = plt.subplots(2, 2, figsize=(12, 8)) 
plt.suptitle('Varianza = 0.1', fontsize=16) 

axs[0, 0].plot(t, p_tilde(t), label='p(x)')
axs[0, 0].hist(x_1, bins=50, density=True, alpha=0.7, label='samples')
axs[0, 0].set_title('Inizializzazione a x0 = -10')
axs[0, 0].legend()

axs[0, 1].plot(t, p_tilde(t), label='p(x)')
axs[0, 1].hist(x_2, bins=50, density=True, alpha=0.7, label='samples')
axs[0, 1].set_title('Inizializzazione a x0 = -2')
axs[0, 1].legend()

axs[1, 0].plot(t, p_tilde(t), label='p(x)')
axs[1, 0].hist(x_3, bins=50, density=True, alpha=0.7, label='samples')
axs[1, 0].set_title('Inizializzazione a x0 = 2')
axs[1, 0].legend()

axs[1, 1].plot(t, p_tilde(t), label='p(x)')
axs[1, 1].hist(x_4, bins=50, density=True, alpha=0.7, label='samples')
axs[1, 1].set_title('Inizializzazione a x0 = 10')
axs[1, 1].legend()

for ax in axs.flat:
    ax.set_xlim(-10, 10)
    ax.set_ylim(0, 0.5)
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8)) 
plt.suptitle('Varianza = 2', fontsize=16) 

axs[0, 0].plot(t, p_tilde(t), label='p(x)')
axs[0, 0].hist(y_1, bins=50, density=True, alpha=0.7, label='samples')
axs[0, 0].set_title('Inizializzazione a x0 = -10')
axs[0, 0].legend()

axs[0, 1].plot(t, p_tilde(t), label='p(x)')
axs[0, 1].hist(y_2, bins=50, density=True, alpha=0.7, label='samples')
axs[0, 1].set_title('Inizializzazione a x0 = -2')
axs[0, 1].legend()

axs[1, 0].plot(t, p_tilde(t), label='p(x)')
axs[1, 0].hist(y_3, bins=50, density=True, alpha=0.7, label='samples')
axs[1, 0].set_title('Inizializzazione a x0 = 2')
axs[1, 0].legend()

axs[1, 1].plot(t, p_tilde(t), label='p(x)')
axs[1, 1].hist(y_4, bins=50, density=True, alpha=0.7, label='samples')
axs[1, 1].set_title('Inizializzazione a x0 = 10')
axs[1, 1].legend()
for ax in axs.flat:
    ax.set_xlim(-10, 10)
    ax.set_ylim(0, 0.5)
plt.tight_layout()
plt.show()

In [ ]:
# sm.tsa.acf(x, nlags=5)
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

tsaplots.plot_acf(x_1, lags=20, ax=axs[0, 0])
axs[0, 0].set_title('ACF x_1 (x0 = -10)')

tsaplots.plot_acf(x_2, lags=20, ax=axs[0, 1])
axs[0, 1].set_title('ACF x_2 (x0 = -2)')

tsaplots.plot_acf(x_3, lags=20, ax=axs[1, 0])
axs[1, 0].set_title('ACF x_3 (x0 = 2)')

tsaplots.plot_acf(x_4, lags=20, ax=axs[1, 1])
axs[1, 1].set_title('ACF x_4 (x0 = 10)')

plt.tight_layout()
plt.suptitle('Autocorrelazioni per diverse inizializzazioni ( Varianza = 0.1 )', fontsize=16)
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

tsaplots.plot_acf(y_1, lags=20, ax=axs[0, 0])
axs[0, 0].set_title('ACF y_1 (y0 = -10)')

tsaplots.plot_acf(y_2, lags=20, ax=axs[0, 1])
axs[0, 1].set_title('ACF y_2 (y0 = -2)')

tsaplots.plot_acf(y_3, lags=20, ax=axs[1, 0])
axs[1, 0].set_title('ACF y_3 (y0 = 2)')

tsaplots.plot_acf(y_4, lags=20, ax=axs[1, 1])
axs[1, 1].set_title('ACF y_4 (y0 = 10)')

plt.tight_layout()
plt.suptitle('Autocorrelazioni per diverse inizializzazioni ( Varianza = 2 )', fontsize=16)
plt.subplots_adjust(top=0.9)
plt.show()

### Problem 4: ADSAI football matches
Over the years, the PhD students of ADSAI have kept track of the results of their evening five-a-side football matches. Since it’s difficult to always organize teams with the same players, the teams were formed on a rotating basis, each time with different players.  
The names of our champions have been anonymized using numbers from 1 to 62.  
In the dataset located at `data/ADSAI_football.csv` in the GitHub repository, you will find the following columns:
- **Team A**: IDs of the players who played in Team A in that match;  
- **Team B**: same as above, for Team B;  
- **Goal A**: total goals scored by Team A in that match;  
- **Goal B**: same as above, for Team B.



Your goal is to model **the skill of each player** based on information about the team they belonged to and the overall result achieved by that team.



The model to implement consists in the following structure: 
- $ \theta = (\theta_1, \theta_2, \dots, \theta_{62}) \in \mathbb{R} $: players' skills.
- $M=54$: number of matches.
- For each match $ i = 1, \dots, M=54$:
  - $ A_i \subset \{1, \dots, N=62\} $: set of players IDs of team A in match $ i $.
  - $ B_i \subset \{1, \dots, N=62\} $: set of players IDs of team B in match $ i $.
  - $ y_i \in \mathbb{Z} $: observed outcome, i.e. goal difference between the two teams, defined as $ (\text{goal}_A - \text{goal}_B) $ in match $ i $.

*(in this exercise, you are asked to follow the proposal of Karlis and Ntzoufras approach, that focuses on the goal difference in place of the individual goal counts of each team!)*



The model is defined as follows:

- The "strength" of Team A, defined as the sum of the individual players' skills $\theta_j$ composing the team in match $i = 1, \dots, M=54$:  
  $$
  s_A^{(i)} = \sum_{j \in A_i} \theta_j
  $$

- The "strength" of Team B, defined similarly as the sum of the individual players' skills $\theta_j$ composing the team in match $i = 1, \dots, M=54$:  
  $$
  s_B^{(i)} = \sum_{j \in B_i} \theta_j
  $$

Specifically, the observed goal difference in match $i$ is modeled using a Skellam distribution:  
$$
y_i \sim \text{Skellam}(\lambda_A^{(i)}, \lambda_B^{(i)}), \quad \text{where} \quad \lambda_A^{(i)} = \exp(s_A^{(i)}), \quad \lambda_B^{(i)} = \exp(s_B^{(i)})
$$

The **Skellam distribution** models the difference between two independent random variables:  
$$
\text{Skellam}(\lambda_A, \lambda_B) = \text{Poisson}(\lambda_A) - \text{Poisson}(\lambda_B)
$$
It is formally defined as:  
$$
\text{Skellam}(k; \lambda_A, \lambda_B) = e^{-(\lambda_A + \lambda_B)} \left( \frac{\lambda_A}{\lambda_B} \right)^{k/2} I_{|k|}(2 \sqrt{\lambda_A \lambda_B})
$$

for each $ k \in \mathbb{Z} $, and $ I_k $ is the modified Bessel function of the first kind of order $ k $.

$$
I_k(z) = \sum_{m=0}^\infty \frac{1}{m! \, \Gamma(m + k + 1)} \left( \frac{z}{2} \right)^{2m + k}
$$

where $\Gamma$ is the Gamma function.


In [ ]:
#import only relevant quantities as follows:

import pandas as pd
import ast
import torch

def preprocessing_dataset(dataset_path='../data/ADSAI_football.csv'):
    football = pd.read_csv(dataset_path)
    football['Team A'] = football['Team A'].apply(ast.literal_eval)
    football['Team B'] = football['Team B'].apply(ast.literal_eval)

    max_player_id = max(
    max(p for team in football['Team A'] for p in team),
    max(p for team in football['Team B'] for p in team)
    )

    goal_diff = torch.tensor((football['Goal A'] - football['Goal B']).values, dtype=torch.int)

    teams_A = [torch.tensor(team) for team in football['Team A']]
    teams_B = [torch.tensor(team) for team in football['Team B']]

    return teams_A, teams_B, goal_diff, max_player_id


teams_A, teams_B, goal_diff, max_player_id = preprocessing_dataset()
print("teams_A:", teams_A)
print("teams_B:", teams_B)
print("goal_diff:", goal_diff)
print("max_player_id:", max_player_id)



You are asked to: 
1. Find the MAP estimate for $ \theta = (\theta_1, \theta_2, \dots, \theta_{62})$ choosing as `log_prior` function a (log-)Standard Normal distribution and as `log_likelihood` function the (log-)Skellam. Perform optimization using Gradient Descent. Use the MAP estimate to implement a Laplace approximation of the posterior, as done during the lab (reuse the function `compute_hessian(f, w)` of Notebook 6).

**Important Hint:** The Skellam log-likelihood involves the modified Bessel function $I_k(z)$, which is non-differentiable in PyTorch if evaluated via scipy. To preserve differentiability, replace $\log I_k(z)$ with a smooth approximation, such as an asymptotic expansion, to allow gradient-based optimization.

2. **(Useful for the next point, not strictly necessary for the previous one):** Implement your `Skellam` distribution, inheriting from `torch.distributions.Distribution`;
3. Write the Pyro model corresponding to the problem depicted above assuming (again) the `theta` values being distributed initially as a Standard Normal;
4. Perform inference on $ \theta = (\theta_1, \theta_2, \dots, \theta_{62})$ values running a MCMC simulation using the `NUTS` kernel;
5. Compare the `theta` values obtained by these two options using the `performances_evaluation` function given in this notebook.


In [ ]:

# 1. Find the MAP estimate for $ \theta = (\theta_1, \theta_2, \dots, \theta_{62})$ choosing as `log_prior` function a (log-)Standard Normal distribution and as `log_likelihood` function the (log-)Skellam. Perform optimization using Gradient Descent;

from torch.distributions import Normal
import torch
from tqdm.notebook import tqdm
import numpy as np
from scipy.special import iv  # Bessel function of the first kind



def log_I_alpha_asymptotic(z, alpha):
    """
    Approssimazione del logaritmo della funzione di Bessel modificata del primo tipo I_alpha(z) in PyTorch.
    Valida per z → ∞.
    """
    z = z.to(dtype=torch.float32)
    alpha = alpha.to(dtype=torch.float32)

    sqrt_term = torch.sqrt(1 + (alpha**2) / z**2)
    log_prefactor = -0.5 * torch.log(2 * torch.pi * z) - 0.25 * torch.log(1 + (alpha**2) / z**2)
    exponent = -alpha * torch.arcsinh(alpha / z) + z * sqrt_term
    log_correction = torch.log(1 + 1 / (z * sqrt_term))

    return log_prefactor + exponent + log_correction

def log_likelihood(teams_A, teams_B, goal_diff, theta):
    lambda_A_list = []
    lambda_B_list = []


    for A_i, B_i in zip(teams_A, teams_B):
        s_A = theta[A_i].sum()
        s_B = theta[B_i].sum()
        lambda_A = torch.exp(s_A)
        lambda_B = torch.exp(s_B)
    
        lambda_A_list.append(lambda_A)
        lambda_B_list.append(lambda_B)

    # Converti in tensori
    lambda_A = torch.stack(lambda_A_list)
    lambda_B = torch.stack(lambda_B_list)
    
    """
    Approssimazione differenziabile della log-probabilità Skellam.
    """
    z = 2 * torch.sqrt(lambda_A * lambda_B)
    goal_diff = goal_diff.to(lambda_A.device)
    goal_diff_abs = goal_diff.abs()

    log_bessel = log_I_alpha_asymptotic(goal_diff_abs, z)
    log_ratio_term = 0.5 * goal_diff * torch.log((lambda_A) / (lambda_B))
    log_p = - (lambda_A + lambda_B) + log_ratio_term + log_bessel
    return log_p.sum()

def log_prior(theta, prior_std=1.0):
    prior = Normal(loc=0.0, scale=prior_std)
    print(f'theta: {theta}')
    return prior.log_prob(theta).sum()

def log_unnormalized_posterior(teams_A, teams_B, goal_diff, theta):
    return log_likelihood(teams_A, teams_B, goal_diff, theta) + log_prior(theta)

# def loss_function(theta):
loss_function = lambda theta: -log_unnormalized_posterior(teams_A, teams_B, goal_diff, theta)

def gradient_descent_optimization(loss_function, lr: float, n_iter: int, initial_guess: torch.Tensor) -> np.ndarray:
    w = torch.tensor(initial_guess, dtype=torch.float32, requires_grad=True)
    print(f"Initial guess 1: {w}")
    for _ in tqdm(range(n_iter)):
        print(f"Iteration {_+1}")
        print(f"Initial guess 2: {w}")
        loss = loss_function(w)
        loss.backward()
        with torch.no_grad():
            w -= lr * w.grad
            print(f"Initial guess 3: {w}")
        w.grad = None
    print(f"loss={ loss.item():.4g}")
    return w.cpu().detach().numpy()

def compute_hessian(f, w):
    w.requires_grad = True
    grad = torch.autograd.grad(f(w), w, create_graph=True)[0]
    hessian = torch.zeros((w.numel(), w.numel()))
    for i in range(w.numel()):
        hessian[i] = torch.autograd.grad(grad[i], w, retain_graph=True)[0]
    w.requires_grad = False
    return hessian

hessian = compute_hessian(f=lambda w : log_unnormalized_posterior(X=X_tensor, y=y_tensor, w=w), w=torch.tensor(w_map, dtype=torch.float32))

# here we want to obtain 
sampled_theta = Normal(loc=0.0, scale=1.0).sample((max_player_id + 1,))
theta_MAP = gradient_descent_optimization(loss_function = loss_function, lr=1e-2, n_iter=1000, initial_guess=sampled_theta)
posterior_cov = torch.inverse(-hessian).numpy()


Initial guess 1: tensor([ 1.1294, -0.8721,  1.9199,  1.5140,  1.6632,  1.3644,  0.0821, -0.5383,
         0.3651,  0.6609, -1.1781,  2.0489,  1.4470, -0.6436, -0.6201, -0.7465,
         2.1286,  0.6165, -0.3305,  0.2446,  0.7473,  1.1628,  0.5824, -0.3945,
         0.4193, -0.3582,  1.1578, -0.9235,  0.1165, -1.0924,  1.7024, -0.4154,
         0.7141, -1.4859, -0.2592,  0.5110,  1.7576,  0.8479, -1.3874, -1.2181,
         1.3761,  1.0408, -0.3430, -0.6162,  0.7718,  1.3587,  0.5252, -0.9933,
        -0.7737, -0.0458,  1.1904,  1.0166,  0.1246, -0.5711, -1.1542, -0.0062,
        -0.4220,  2.3039, -1.3311,  0.5554,  0.3187, -0.0488, -0.8406],
       requires_grad=True)


C:\Users\tomma\AppData\Local\Temp\ipykernel_26588\1581486452.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w = torch.tensor(initial_guess, dtype=torch.float32, requires_grad=True)


  0%|          | 0/1000 [00:00<?, ?it/s]

Iteration 1
Initial guess 2: tensor([ 1.1294, -0.8721,  1.9199,  1.5140,  1.6632,  1.3644,  0.0821, -0.5383,
         0.3651,  0.6609, -1.1781,  2.0489,  1.4470, -0.6436, -0.6201, -0.7465,
         2.1286,  0.6165, -0.3305,  0.2446,  0.7473,  1.1628,  0.5824, -0.3945,
         0.4193, -0.3582,  1.1578, -0.9235,  0.1165, -1.0924,  1.7024, -0.4154,
         0.7141, -1.4859, -0.2592,  0.5110,  1.7576,  0.8479, -1.3874, -1.2181,
         1.3761,  1.0408, -0.3430, -0.6162,  0.7718,  1.3587,  0.5252, -0.9933,
        -0.7737, -0.0458,  1.1904,  1.0166,  0.1246, -0.5711, -1.1542, -0.0062,
        -0.4220,  2.3039, -1.3311,  0.5554,  0.3187, -0.0488, -0.8406],
       requires_grad=True)
theta: tensor([ 1.1294, -0.8721,  1.9199,  1.5140,  1.6632,  1.3644,  0.0821, -0.5383,
         0.3651,  0.6609, -1.1781,  2.0489,  1.4470, -0.6436, -0.6201, -0.7465,
         2.1286,  0.6165, -0.3305,  0.2446,  0.7473,  1.1628,  0.5824, -0.3945,
         0.4193, -0.3582,  1.1578, -0.9235,  0.1165, -1.0924,  1.

ValueError: Expected value argument (Tensor of shape (63,)) to be within the support (Real()) of the distribution Normal(loc: 0.0, scale: 1.0), but found invalid values:
tensor([ 1.1181, -1.2424,  1.8166,  1.3224,  1.5175,  0.9957,     nan, -0.6131,
         0.3663,  0.2992,     nan,  0.9145,  1.0775, -0.6388, -0.7086,     nan,
            nan,     nan,     nan, -0.1341,  0.4444,     nan,     nan,     nan,
            nan, -0.5265,     nan,     nan, -0.2397,     nan,     nan, -0.4401,
         0.5990, -1.8261, -0.3657,     nan,     nan,  0.8249, -1.3649, -1.2272,
            nan, -4.2714, -0.3305,     nan,  0.6839, -0.7833,  0.4398, -1.6362,
        -0.9663, -0.1044,  1.0160,  0.9262,     nan,     nan, -1.1638, -0.1023,
        -0.5139,  1.7641, -1.2669,  0.4907,  0.2564, -0.3907, -0.9593],
       requires_grad=True)

In [ ]:
#Visualize the marginal distribution of some thetas

import matplotlib.pyplot as plt
from torch.distributions import MultivariateNormal

# Sample from the full posterior
mvn = MultivariateNormal(loc=theta_MAP, covariance_matrix=torch.from_numpy(posterior_cov).float())
posterior_samples = mvn.sample((1000,)) 

# Indices of thetas you want to visualize
selected_indices = [0, 1, 2, 20]  # Change these to the indices you're interested in

# Plot histograms
plt.figure(figsize=(10, 6))
for i, idx in enumerate(selected_indices):
    plt.subplot(2, 2, i + 1)
    plt.hist(posterior_samples[:, idx].numpy(), bins=40, density=True, alpha=0.7)
    plt.title(f"Posterior of $\\theta_{{{idx}}}$")
    plt.xlabel("Value")
    plt.ylabel("Density")
plt.tight_layout()
plt.show()


In [ ]:
# 2. Implement your `Skellam` distribution, inheriting from `torch.distributions.Distribution`;

import torch
import pyro
import pyro.distributions as dist
from torch.distributions import constraints
from torch.distributions.distribution import Distribution
import scipy.special


class Skellam(Distribution):
    arg_constraints = {
        "lambdaA": constraints.positive,
        "lambdaB": constraints.positive
    }
    support = constraints.dependent
    has_rsample = False

    def __init__(self, lambdaA, lambdaB, validate_args=None):
        self.lambdaA, self.lambdaB = torch.broadcast_tensors(lambdaA, lambdaB)
        batch_shape = self.lambdaA.shape
        super().__init__(batch_shape, validate_args=validate_args)

    def sample(self, sample_shape=torch.Size()):
        shape = self._extended_shape(sample_shape)
        A = torch.poisson(self.lambdaA.expand(shape))
        B = torch.poisson(self.lambdaB.expand(shape))
        return A - B

    def log_prob(self, value):
        k = value
        lamA = self.lambdaA
        lamB = self.lambdaB

        # Calcolo Bessel con SciPy
        iv_numpy = scipy.special.iv(k.abs().cpu().numpy(), 2 * (lamA * lamB).sqrt().cpu().numpy())
        iv_term = torch.from_numpy(iv_numpy).to(lamA.device)

        iv_term = torch.clamp(iv_term, min=1e-20)
        log_iv = torch.log(iv_term)

        return -(lamA + lamB) + 0.5 * k * torch.log(lamA / lamB) + log_iv

In [ ]:
 
# 3. Write the Pyro model corresponding to the problem depicted above assuming (again) the `theta` values being distributed initially as a Standard Normal;
def model(observations: torch.Tensor | None = None, num_matches: int | None = None):
    
    if observations is not None:
        num_matches = len(observations)
    
    # theta: abilità dei giocatori (Standard Normal prior)
    theta = pyro.sample("theta", dist.Normal(0., 1.).expand([max_player_id + 1]).to_event(1))

    with pyro.plate("matches", num_matches):
        s_A = torch.stack([theta[team].sum() for team in teams_A])
        s_B = torch.stack([theta[team].sum() for team in teams_B])

        lambda_A = torch.exp(s_A)
        lambda_B = torch.exp(s_B)

        goal_diff = pyro.sample("obs", Skellam(lambda_A, lambda_B), obs=observations)
    
    return {'obs': goal_diff}


# 2: plot the corresponding graphical model
print(goal_diff)

# observations = goal_diff
# pyro.render_model(model, model_kwargs= {'num_matches': 10}, render_distributions=True)


In [ ]:
samples_1 = pd.DataFrame(model(num_matches=10))    
samples_1
samples_2 = pd.DataFrame(model(observations=observations))    
samples_2

In [ ]:
 
# 4. Perform inference on $ \theta = (\theta_1, \theta_2, \dots, \theta_{62})$ values running a MCMC simulation using the `NUTS` kernel;

from pyro.infer import MCMC, NUTS
kernel = NUTS(model) 
mcmc = MCMC(kernel, num_samples=1000, warmup_steps=500)
mcmc.run(goal_diff, teams_A, teams_B, max_player_id)
mcmc.summary()

theta_MCMC = ...

In [ ]:

# 5. Compare the `theta_*` values obtained by these two options using the `performances_evaluation` function given in this notebook.

import numpy as np
import torch

def predict_goal_diff_skellam(teamA_ids, teamB_ids, theta, n_sim=10_000, posterior_cov=None):
    """
    Predicts the goal difference (Skellam distribution) between two teams using theta.

    Args:
        teamA_ids (list): Indices of the players in team A.
        teamB_ids (list): Indices of the players in team B.
        theta (torch.Tensor): Tensor containing the skill levels of the players.
        n_sim (int): Number of simulations to run.

    Returns:
        float: Mean predicted goal difference (A - B).
    """

    # Calculate the total strength for each team (sum of player skill levels)
    sA = theta[teamA_ids].sum()
    sB = theta[teamB_ids].sum()

    # Lambda parameters for Poisson distribution (expected goals)
    lam_A = torch.exp(sA).item()  # Team A's expected goal rate
    lam_B = torch.exp(sB).item()  # Team B's expected goal rate

    # Simulate goals for each team using Poisson distribution
    goals_A = np.random.poisson(lam_A, size=n_sim)
    goals_B = np.random.poisson(lam_B, size=n_sim)

    # Calculate the difference in goals (A - B)
    diff = goals_A - goals_B

    # Return the mean predicted difference
    return diff.mean()


def predict_goal_diff_laplace(teamA_ids, teamB_ids, theta_map, cov_matrix, n_sim=1000):
    # Combine all relevant indices
    all_ids = torch.tensor(sorted(set(teamA_ids) | set(teamB_ids)))

    # Extract subvector of means (mu_A and mu_B)
    theta_sub = theta_map[all_ids].float()

    # Extract submatrix of covariances
    cov_sub = cov_matrix[np.ix_(all_ids, all_ids)]  # numpy version
    cov_sub = torch.from_numpy(cov_sub).float()

    # Create multivariate normal from posterior
    mvn = MultivariateNormal(loc=theta_sub, covariance_matrix=cov_sub)

    # Sample from posterior
    theta_samples = mvn.sample((n_sim,))  # shape: (n_sim, len(all_ids))

    # Compute strength for each team in each sample
    teamA_len = len(teamA_ids)
    sA = theta_samples[:, :teamA_len].sum(dim=1)
    sB = theta_samples[:, teamA_len:].sum(dim=1)

    # Expected goals via Poisson lambdas
    lam_A = torch.exp(sA)
    lam_B = torch.exp(sB)

    # Sample goals
    goals_A = torch.poisson(lam_A)
    goals_B = torch.poisson(lam_B)

    # Goal differences
    diff = goals_A - goals_B
    return diff.mean()


def evaluate_performance(theta_MAP, theta_MCMC, posterior_cov, teams_A, teams_B, goal_diff, n_sim=10_000):
    """
    Evaluates the performance of different sets of theta values by comparing the Mean Bias Error (MBE)
    and Mean Absolute Error (MAE) between predicted and actual goal differences.

    Args:
        theta_values (list of torch.Tensor): List of different theta values to evaluate.
        teams_A (list of lists): List of player IDs for team A in each match.
        teams_B (list of lists): List of player IDs for team B in each match.
        goal_diff (list): List of actual goal differences (A - B).
        n_sim (int): Number of simulations to run for each set of theta values.

    Returns:
        dict: Dictionary with MBE and MAE for each set of theta values.
    """
    results = {}

    # Loop through each set of theta values
 
    predicted_diffs_MAP = []  # List to store predicted goal differences
    predicted_diffs_MCMC = []
    actual_diffs = goal_diff  # Actual goal differences

    # Simulate the match results for each game
    for i in range(len(goal_diff)):
        teamA_ids = teams_A[i]
        teamB_ids = teams_B[i]

        # Predict the goal difference 
        # Use the Laplace approximation if posterior_cov is provided
        
        predicted_MAP = predict_goal_diff_laplace(teamA_ids, teamB_ids, theta_MAP, posterior_cov, n_sim)
        predicted_MCMC = predict_goal_diff_skellam(teamA_ids, teamB_ids, theta_MCMC, n_sim)
        
        predicted_diffs_MAP.append(predicted_MAP)
        predicted_diffs_MCMC.append(predicted_MCMC)

    # Convert lists to numpy arrays for easier calculations
    predicted_diffs_MAP = np.array(predicted_diffs_MAP)
    predicted_diffs_MCMC = np.array(predicted_diffs_MCMC)
    actual_diffs = np.array(actual_diffs)

    # Compute MAE and MBE
    mae_MAP = np.mean(np.abs(predicted_diffs_MAP - actual_diffs))  # Mean Absolute Error
    mbe_MAP = (predicted_diffs_MAP - actual_diffs).mean()  # Mean Bias Error
    mae_MCMC = np.mean(np.abs(predicted_diffs_MCMC - actual_diffs))  # Mean Absolute Error
    mbe_MCMC = (predicted_diffs_MCMC - actual_diffs).mean()  # Mean Bias Error

    # Store results for the current theta set
    results = {'MAE_MAP': mae_MAP, 'MBE_MAP': mbe_MAP,
              'MAE_MCMC': mae_MCMC, 'MBE_MCMC': mbe_MCMC}

    return results


# Evaluate the performance of different theta values on the same dataset used to fix those values
performance_results = evaluate_performance(theta_MCMC, theta_MAP, posterior_cov, teams_A, teams_B, goal_diff)

# Print the results
print("Performance Results:")
print(f"MAP - MAE: {performance_results['MAE_MAP']}, MBE: {performance_results['MBE_MAP']}")
print(f"MCMC - MAE: {performance_results['MAE_MCMC']}, MBE: {performance_results['MBE_MCMC']}")